# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
from pprint import pprint

# Import API key
from api_keys import g_key


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
# Load CSV into dataframe
file = '../output_data/cities.csv'
df = pd.read_csv(file)
df.head()

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,Nikolskoye,75,RU,1580270243,92,59.70,30.79,30.20,11.18
1,Ozinki,100,RU,1580270554,91,51.18,49.68,11.95,13.06
2,Rikitea,100,PF,1580270485,78,-23.12,-134.97,79.79,6.22
3,Vaini,75,TO,1580270072,94,-21.20,-175.20,78.80,5.82
4,Talnakh,100,RU,1580270159,78,69.49,88.40,8.60,6.71


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Configure gmaps using API key
gmaps.configure(api_key=g_key)

# Zip latitudes and longtitudes into tuples and store in a list
locations = list(zip(df['Lat'], df['Lng']))

# Store humidity values
humidity = df['Humidity']

# Set default center and zoom level
fig = gmaps.figure(center=(0,0), zoom_level=2)

# Define heatmap parameters
heatmap_layer = gmaps.heatmap_layer(locations, weights=humidity)

# Add heatmap layers
fig.add_layer(heatmap_layer)

fig


Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [4]:
# Create new dataframe based on my criteria
hotel_df = df[(df['Max Temp'] > 75) & (df['Max Temp'] < 95) & (df['Wind Speed'] < 10) & (df['Cloudiness'] < 10)].copy()

# Display entire dataframe
hotel_df

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
53,San Patricio,0,MX,1580270496,63,19.22,-104.70,76.05,4.29
97,Contamana,0,PE,1580270690,67,-7.33,-75.02,79.79,2.86
136,Colac,0,AU,1580270744,40,-38.33,143.58,86.00,4.00
187,Kapaa,1,US,1580270629,61,22.08,-159.32,78.80,3.36
230,Bonfim,7,BR,1580270861,59,3.08,-59.95,82.99,7.96
237,Kollam,7,IN,1580270870,69,8.88,76.60,82.40,4.70
352,Mount Gambier,0,AU,1580271009,17,-37.83,140.77,89.60,9.17
387,Acapulco de Juarez,5,MX,1580270867,74,16.86,-99.89,78.80,5.82
443,La Macarena,4,CO,1580271129,63,2.18,-73.78,77.49,2.55
539,Cockburn Town,0,TC,1580271207,70,21.46,-71.14,77.67,8.41


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [5]:
# Add column
hotel_df['Hotel Name'] = ''

# Define url and parameters
url = 'https://maps.googleapis.com/maps/api/place/nearbysearch/json'
params = {'location':'-33.65,115.33',
          'radius':5000,
          'key':g_key,
          'keyword':'hotel'}

# Initialize variables
hotels = []
lat = []
lng = []

# Zip latitudes and longtitudes and store in a list
locations = list(zip(hotel_df['Lat'], hotel_df['Lng']))

# Loop through lat, lng pairs, call Google API for hotels, and store first result in list
for i in locations:
    params['location'] = f'{i[0]},{i[1]}'
    jason = requests.get(url, params=params).json()
    
    try:
        hotels.append(jason['results'][0]['name'])
        lat.append(jason['results'][0]['geometry']['location']['lat'])
        lng.append(jason['results'][0]['geometry']['location']['lng'])
    except IndexError:
        hotels.append('None')
        lat.append('N/A')
        lng.append('N/A')

# Update dataframe with hotel names, latitudes, and longtitudes
hotel_df['Hotel Name'] = hotels
hotel_df['Lat'] = lat
hotel_df['Lng'] = lng
hotel_df.head()

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
53,San Patricio,0,MX,1580270496,63,19.1954,-104.684,76.05,4.29,Grand Isla Navidad Resort
97,Contamana,0,PE,1580270690,67,-7.35225,-75.0087,79.79,2.86,Hospedaje Florencia
136,Colac,0,AU,1580270744,40,-38.3379,143.598,86.00,4.00,Otway Gate Motel
187,Kapaa,1,US,1580270629,61,22.0786,-159.315,78.80,3.36,Hotel Coral Reef
230,Bonfim,7,BR,1580270861,59,N/A,N/A,82.99,7.96,None


In [12]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
hotel_df = hotel_df[hotel_df['Hotel Name'] != 'None']
locations = list(zip(hotel_df["Lat"], hotel_df["Lng"]))

In [14]:
# Add marker layer ontop of heat map
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# Display Map
fig

Figure(layout=FigureLayout(height='420px'))